### Targeted information retrieval

We have seen how to parse a page, retrieve without distinction the information of a web page

But, in general, the purpose of scrapping is to automate the collection of targeted information on the web


In [7]:
from bs4 import BeautifulSoup
import requests


Let's say I want to scrape the description of the latest movies released in theaters

So I go to the allo ciné site and try to find the tags that will give me links to the specific pages of these films to get their summaries.

#### Recovery of the url of the pages of films freshly released in the cinema

In [9]:
url='http://www.allocine.fr/'
r = requests.get(url)
print(url, r.status_code)
soup = BeautifulSoup(r.content,'lxml')
soup

http://www.allocine.fr/ 200


<!DOCTYPE html>
<html>
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="index,follow" name="robots"/>
<title>AlloCiné : Cinéma, Séries TV, BO de films et séries, Vidéos, DVD et VOD</title>
<meta content="" name="keywords"/>
<meta content="noarchive" name="Googlebot"/>
<meta content="global" name="distribution"/>
<meta content="AlloCine" name="author"/>
<meta content="France" name="country"/>
<meta content="48.87078;2.30447" name="geo.position"/>
<meta content="FR" name="geo.country"/>
<meta content="48.87078;2.30447" name="ICBM"/>
<meta content="#FECC00" name="theme-color"/>
<meta content="AlloCiné, le site de référence du cinéma et des séries tv ! Découvrez notre recherche d'horaires de films, le programme tv de vos séries préférées, l'actualité ciné et séries, les émissions AlloCiné, les dernières 

On my web browser (Chrome, mozilla...) I use the "inspect" function (right click / inspect) and I drag my mouse to the areas of the page that interest me. At the same time I will see the html script move to the instructions of the html script in question. 

That's how I do it to find the tags I'm interested in.

I notice that the relative link of the web page specific to the new movie on the poster is stored in these tags:

a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=235582.html" title="Le Grand Bain">Le Grand Bain</a


In [6]:
for p in soup.find_all('a'):
    print (p.text)

Bohemian Rhapsody
Mowgli
Game of Thrones
The Walking Dead
séances cinéma
news
dossiers
émissions AlloCiné
dernières bandes-annonces

            Cinéma
        

            Séries
        
 B.O.
        

            DVD
        

            VOD
        
 Netflix
        
Avant-premières
Agenda des sorties
Films pour enfants à l'affiche
Meilleurs films
Box Office
Meilleurs films 2017
Bandes-annonces de films
BA des films prochainement au cinéma
Tous les films pour enfants
Tous les films
Sorties de la semaine
Séances
Bande-annonce
Le Grand Bain
Séances
Bande-annonce
Chair de poule 2 : Les Fantômes d'Halloween
Séances
Bande-annonce
Halloween
Séances
Bande-annonce
Jean-Christophe & Winnie
Séances
Bande-annonce
Cold War
Séances
Bande-annonce
Quién te cantará
Séances
Bande-annonce
La Tendre indifférence du monde
Séances
Bande-annonce
Bamse au pays des voleurs
Séances
La Fille du Far-West (Met - Pathé Live)
Séances
Die Walküre (Royal Opera House)
Dilili à Paris
Alad'2
Yéti & Compagnie
Veno

This time the site is more difficult to "extract", let's use additional parameters to this search function, find_all, much more specific than the one currently used

In [12]:
# In addition to the tag a, well identified, we notice some additional 
# information such as the value of the class variable of these identical tags
for elem in soup.find_all('a',attrs={"class" :"meta-title meta-title-link"}):
    print(elem)

<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=235582.html" title="Le Grand Bain">Le Grand Bain</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=247007.html" title="Chair de poule 2 : Les Fantômes d'Halloween">Chair de poule 2 : Les Fantômes d'Halloween</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=238622.html" title="Halloween">Halloween</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=251837.html" title="Jean-Christophe &amp; Winnie">Jean-Christophe &amp; Winnie</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=252203.html" title="Cold War">Cold War</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=253056.html" title="Quién te cantará">Quién te cantará</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=264018.html" title="La Tendre indifférence du monde">La Tendre indifférence du monde</a>
<a class="meta-title meta

#### Recovery of href

We have noticed the presence of href links to the pages that interest us. Let's go get them back:

In [16]:
for elem in soup.find_all('a',attrs={"class" :"meta-title meta-title-link"}):
    print(elem.get('href'))
    # return a list

Le Grand Bain
Chair de poule 2 : Les Fantômes d'Halloween
Halloween
Jean-Christophe & Winnie
Cold War
Quién te cantará
La Tendre indifférence du monde
Bamse au pays des voleurs
La Fille du Far-West (Met - Pathé Live)
Die Walküre (Royal Opera House)
Dilili à Paris
Alad'2
Yéti & Compagnie
Venom
Le Jeu
Le Flic de Belleville
A Star Is Born
La Prophétie de l'horloge
Élite
Le Bureau des Légendes
The Haunting of Hill House
La Casa De Papel
Tell Me a Story
Homecoming
Nightflyers
Ad Vitam


Can you retrieve the titles for me via the search for "title" in the items of the previous list?

#### Get summary

Let's start by building the url that we will use to retrieve the summaries

Start by putting the href values in a list of links


In [10]:
links=[]
for elem in soup.find_all('a',attrs={"class" :"meta-title meta-title-link"}):
    # Tout simplement
    links.append(elem.get('href'))
    # Je range tout cela dans une liste 
links

['/film/fichefilm_gen_cfilm=260395.html',
 '/film/fichefilm_gen_cfilm=263492.html',
 '/film/fichefilm_gen_cfilm=178703.html',
 '/film/fichefilm_gen_cfilm=255745.html',
 '/film/fichefilm_gen_cfilm=260671.html',
 '/film/fichefilm_gen_cfilm=242028.html',
 '/film/fichefilm_gen_cfilm=260353.html',
 '/film/fichefilm_gen_cfilm=260482.html',
 '/film/fichefilm_gen_cfilm=270311.html',
 '/film/fichefilm_gen_cfilm=261843.html',
 '/film/fichefilm_gen_cfilm=230141.html',
 '/film/fichefilm_gen_cfilm=242959.html',
 '/film/fichefilm_gen_cfilm=264918.html',
 '/film/fichefilm_gen_cfilm=126922.html',
 '/film/fichefilm_gen_cfilm=265032.html',
 '/film/fichefilm_gen_cfilm=141110.html',
 '/film/fichefilm_gen_cfilm=264206.html',
 '/film/fichefilm_gen_cfilm=259452.html',
 '/series/ficheserie_gen_cserie=20413.html',
 '/series/ficheserie_gen_cserie=22114.html',
 '/series/ficheserie_gen_cserie=18968.html',
 '/series/ficheserie_gen_cserie=22964.html',
 '/series/ficheserie_gen_cserie=23870.html',
 '/series/ficheseri

The absolute url of the searched film pages is built in this form: http://www.allocine.fr/film/fichefilm_gen_cfilm=243835.html

It is therefore necessary to repeat the previous list and build the absolute urls for our search

It's up to you to play

nb: Do not take the links for the series

In [11]:
links_movie=['http://www.allocine.fr'+ elem for elem in links if 'film' in elem]
links_movie

['http://www.allocine.fr/film/fichefilm_gen_cfilm=260395.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=263492.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=178703.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=255745.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=260671.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=242028.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=260353.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=260482.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=270311.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=261843.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=230141.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=242959.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=264918.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=126922.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=265032.html',
 'http://www.allocine.fr/film/fichefilm_

Finally, on each page, the title and synopsis must be retrieved. Let's try for a movie, the first of the link list

In [13]:
url=links_movie[0]
r = requests.get(url)
print(url, r.status_code)
soup = BeautifulSoup(r.content,'lxml')
soup

http://www.allocine.fr/film/fichefilm_gen_cfilm=260395.html 200


/home/ludo/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/ludo/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


<!DOCTYPE html>
<html>
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="index,follow" name="robots"/>
<title>Tanguy, le retour - film 2018 - AlloCiné</title>
<meta content="" name="keywords"/>
<meta content="noarchive" name="Googlebot"/>
<meta content="global" name="distribution"/>
<meta content="AlloCine" name="author"/>
<meta content="France" name="country"/>
<meta content="48.87078;2.30447" name="geo.position"/>
<meta content="FR" name="geo.country"/>
<meta content="48.87078;2.30447" name="ICBM"/>
<meta content="#FECC00" name="theme-color"/>
<meta content='Tanguy, le retour est un film réalisé par Étienne Chatiliez avec André Dussollier, Sabine Azéma. Synopsis : 16 ans plus tard, Tanguy, qui a maintenant 44 ans, revient chez ses parents avec sa fille Zhu sous le bras car Meï Lin l’a quitté. Catast

For title 

div class="titlebar-title titlebar-title-lg">Le Grand Bain</div

For the synopsis

div class="content-txt " itemprop="description"
    
              C’est dans les couloirs de leur piscine municipale que Bertrand, Marcus, Simon, Laurent, Thierry et les autres s’entraînent sous l’autorité toute relative de Delphine, ancienne gloire des bassins. Ensemble, ils se sentent libres et utiles. Ils vont mettre toute leur énergie dans une discipline jusque-là propriété de la gent féminine : la natation synchronisée. Alors, oui c’est une idée plutôt bizarre, mais ce défi leur permettra de trouver un sens à leur vie...
    
      </div

In [30]:
for elem in soup.find_all('div',attrs={"class" :"titlebar-title titlebar-title-lg"}):
    # Tout simplement
    print(elem.text)
    
for elem in soup.find_all('div',attrs={"class" :"content-txt "}):
    # Tout simplement
    print(elem.text)

Le Grand Bain

    
              C’est dans les couloirs de leur piscine municipale que Bertrand, Marcus, Simon, Laurent, Thierry et les autres s’entraînent sous l’autorité toute relative de Delphine, ancienne gloire des bassins. Ensemble, ils se sentent libres et utiles. Ils vont mettre toute leur énergie dans une discipline jusque-là propriété de la gent féminine : la natation synchronisée. Alors, oui c’est une idée plutôt bizarre, mais ce défi leur permettra de trouver un sens à leur vie...
    
      


1) Automate this script for the entire list

2) Put the information in three lists (film_links, title, synopsis)

3) Create a dataframe that includes these three informations in three associated columns

4) Save this dataframe in a csv file

And here's your first real scrap, you're real hackers now.

In [14]:
import time
import random
from random import randint

title=[]
synopsis=[]

for link in links_movie:
    
    url=link
    # I slow down the frequency of requests to avoid being identified and therefore ban from the site
    time.sleep(random.uniform(1.0, 2.0))
    r = requests.get(url)
    print(url, r.status_code)
    soup = BeautifulSoup(r.content,'lxml')
    
    
    for elem in soup.find_all('div',attrs={"class" :"titlebar-title titlebar-title-lg"}):
        title.append(elem.text.strip())

    for elem in soup.find_all('div',attrs={"class" :"content-txt "}):
        synopsis.append(elem.text.strip())
        
# I check the length of the lists before creating the df
len(title),len(synopsis),len(links_movie)

http://www.allocine.fr/film/fichefilm_gen_cfilm=260395.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=263492.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=178703.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=255745.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=260671.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=242028.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=260353.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=260482.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=270311.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=261843.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=230141.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=242959.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=264918.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=126922.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=265032.html 200
http://www.allocine.fr/film/fichefilm_ge

(18, 18, 18)

In [40]:
import pandas as pd
df=pd.DataFrame({'Titre':titre})
df['synopsis']=synopsis
df['liens']=liens_film

In [41]:
df.to_csv('../data/allo_cine.csv', index=False)